# Wrangling data for in-the-wild robot experiments

## Preamble

In [11]:
#install packages
#!pip install gspread==2.1.1
#!pip install gspread-dataframe==2.1.0

import pandas as pd
import numpy as np
import os
import gspread

from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.colab import auth
from google.auth import default
from oauth2client.client import GoogleCredentials
creds, _ = default()
auth.authenticate_user()  # verify your account to read files which you have access to. Make sure you have permission to read the file!
gc = gspread.authorize(creds)

from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/Y4/S2/FIT4441/Ethics/Survey")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
[f for f in os.listdir("raw") if f.endswith(".gsheet")]

['Survey (OC-SORT) (Responses).gsheet',
 'Interviews.gsheet',
 'Count data.gsheet',
 'Survey (Default) (Responses) (1).gsheet',
 'Survey (Default) (Responses).gsheet',
 'Count data W1.gsheet']

In [13]:
# Load data

urls = ["https://docs.google.com/spreadsheets/d/1xOdw0eIpWAbxceh9Ax6KXsbPLfTsJao7RHPMHcld_b8/edit?usp=drive_link",
     "https://docs.google.com/spreadsheets/d/1SKUq6QA8j_oFKPnruL_cKniRmORzTktBbhVztEdx_MI/edit?usp=drive_link",
     "https://docs.google.com/spreadsheets/d/13cP7TLezsEGIPFm2mKwlaH1bfHy09seyJ9SO3yJX1mE/edit?usp=drive_link"]

# Read sheets
observations = gc.open_by_url(urls[0])
ocs = gc.open_by_url(urls[2])
defaults = gc.open_by_url(urls[1])

# Read sheets as df
observations_oc = pd.DataFrame(observations.get_worksheet(0).get_all_records()).convert_dtypes()
observations_default = pd.DataFrame(observations.get_worksheet(1).get_all_records()).convert_dtypes()
default = pd.DataFrame(defaults.get_worksheet(0).get_all_records()).reset_index(drop=True).replace("",np.nan).dropna().convert_dtypes()
oc = pd.DataFrame(ocs.get_worksheet(0).get_all_records()).reset_index(drop=True).replace("",np.nan).dropna(how="all").convert_dtypes()

## Cleaning Observation data

For observation data, I forgot that I can just fill NA's after data collection, so I made many default entries for rows, even though I didn't collect data for them. Hence, our data is going to look pretty inflated:

In [14]:
print("Default shape:", observations_default.shape)
print("OC shape:", observations_oc.shape)

Default shape: (199, 13)
OC shape: (425, 13)


One way to tell is to look at their group numbers. The default group number is always 0. The only time when this is valid is when it's the first row in our data. Hence, we can remove every other row featuring **Group**==0:

In [15]:
def delete_group_0s(df):
    # Get every row with Group = 0, only grab from 2nd to last rows, and save their indices:
    idx = df[df.Group==0][1:].index
    # Drop those indices and return
    return df.drop(idx)

# Drop extra rows for observation data:
observations_default = delete_group_0s(observations_default)
observations_oc = delete_group_0s(observations_oc)

Let's examine the shape of our final observation data frames:

In [16]:
print("Default shape:", observations_default.shape)
print("OC shape:", observations_oc.shape)

Default shape: (138, 13)
OC shape: (425, 13)


During sessions for OC-SORT, I've mistakened the column of **false_alarm** for **Survey** and vice versa. So, let's switch it around:

In [17]:
observations_oc["false_alarm"], observations_oc["Survey"] = observations_oc["Survey"], observations_oc["false_alarm"]
observations_oc.head()

,Time,Group,Participant,Success,Invested,Photo,Emotion,false_alarm,Notes,Survey,I1,I2,I3
0,27/07/2023 12:02:22,0,0,0,0,0,N,,,0,,,
1,27/07/2023 12:02:34,1,0,0,0,0,N,,,0,,,
2,27/07/2023 12:02:40,2,0,0,0,0,N,,,0,,,
3,27/07/2023 12:02:52,3,0,0,0,0,N,,,0,,,
4,27/07/2023 12:04:15,4,0,0,0,0,N,,,0,,,


Since **false_alarm** is the only boolean with NA, we'll fill it with 0 as the default:

In [18]:
observations_oc.false_alarm.fillna(0, inplace=True)
observations_oc.head()

,Time,Group,Participant,Success,Invested,Photo,Emotion,false_alarm,Notes,Survey,I1,I2,I3
0,27/07/2023 12:02:22,0,0,0,0,0,N,,,0,,,
1,27/07/2023 12:02:34,1,0,0,0,0,N,,,0,,,
2,27/07/2023 12:02:40,2,0,0,0,0,N,,,0,,,
3,27/07/2023 12:02:52,3,0,0,0,0,N,,,0,,,
4,27/07/2023 12:04:15,4,0,0,0,0,N,,,0,,,


## Cleaning survey data

There's a shape mismatch between **oc** and **default**:

In [19]:
print("OC-SORT survey shape:", oc.shape)
print("Default survey shape:", default.shape)

OC-SORT survey shape: (8, 17)
Default survey shape: (9, 14)


This is because during the pilot study, I included interview questions in the survey response for OC-SORT, which was a bad idea. Let's make it so that has the same columns as **default**:

In [20]:
oc = oc[default.columns.tolist()]
print("New oc shape:", oc.shape)

New oc shape: (8, 14)


A major issue in the survey data is that they're all strings:

In [21]:
oc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8 entries, 3 to 10
Data columns (total 14 columns):
 #   Column                                                                             Non-Null Count  Dtype 
---  ------                                                                             --------------  ----- 
 0   Timestamp                                                                          8 non-null      string
 1   Are you 18 years or older?                                                         8 non-null      string
 2   A1                                                                                 8 non-null      Int64 
 3   A2                                                                                 8 non-null      Int64 
 4   A3                                                                                 8 non-null      Int64 
 5   A4                                                                                 8 non-null      Int64 
 6   A5   

We shall impute Yes/No with 1's and 0's to faciliate data vis later on. We'll also convert sections **A** and **B** to integers:

In [22]:
# Convert string from sections A and B to int
oc = oc.apply(pd.to_numeric, errors="ignore", downcast='integer')
default = default.apply(pd.to_numeric, errors="ignore", downcast='integer')

In [23]:
default.head()

,Timestamp,Are you 18 years or older?,A1,A2,A3,A4,A5,B1,B2,B3,C1. Are you satisfied with Pepper's ability to follow you?,C2. Would you be comfortable with Pepper serving you as a waiter in a restaurant?,D1. Have you ever interacted with a robot before?,D2. Have you previously participated in this experiment?
1,25/07/2023 12:32:14,Yes,4,3,4,3,3,4,2,3,4,3,No,No
2,25/07/2023 12:41:12,Yes,4,3,4,4,5,4,5,4,4,4,Yes,Yes
3,25/07/2023 12:55:22,Yes,3,3,3,3,3,3,1,2,3,3,Yes,No
4,25/07/2023 13:33:15,Yes,3,4,3,4,4,5,1,4,4,2,No,No
5,25/07/2023 13:57:52,Yes,4,3,4,3,4,5,5,4,4,5,Yes,No


In [24]:
m = {"Yes":1,"No":0}
cls = [1,-2,-1]

# Mapping "Yes" to 1, and "No" to 0:
oc.iloc[:,cls] = oc.iloc[:,cls].apply(lambda c: c.map(m))
default.iloc[:,cls] = default.iloc[:,cls].apply(lambda c: c.map(m))

<ipython-input-24-f37083e286f6>:5: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  oc.iloc[:,cls] = oc.iloc[:,cls].apply(lambda c: c.map(m))
<ipython-input-24-f37083e286f6>:6: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  default.iloc[:,cls] = default.iloc[:,cls].apply(lambda c: c.map(m))


In [25]:
oc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8 entries, 3 to 10
Data columns (total 14 columns):
 #   Column                                                                             Non-Null Count  Dtype 
---  ------                                                                             --------------  ----- 
 0   Timestamp                                                                          8 non-null      object
 1   Are you 18 years or older?                                                         8 non-null      int64 
 2   A1                                                                                 8 non-null      Int8  
 3   A2                                                                                 8 non-null      Int8  
 4   A3                                                                                 8 non-null      Int8  
 5   A4                                                                                 8 non-null      Int8  
 6   A5   

Conversion success!


Also, in section *B*, the scales are inconsistent as *B1* doesn't align with *B2* and *B3*. We decided to reverse the order of *B2* and *B3* such that higher numbers represent a higher perceived safety:

In [26]:
bs = ["B2","B3"]
oc.loc[:,bs] = (oc[bs]-6).abs()
default.loc[:,bs] = (default[bs]-6).abs()

<ipython-input-26-d6d6bf3969f1>:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  oc.loc[:,bs] = (oc[bs]-6).abs()
<ipython-input-26-d6d6bf3969f1>:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  default.loc[:,bs] = (default[bs]-6).abs()


## Checking data integrity

It's likely that I might've made some mistakes during data collection. Let's fix some of them!

### Is Group increasing?

**Group** was collected by incrementing the previous value by 1 for each new group/person that has walked in front of the robot. Since this was done by hand, it's likely that I might've made mistakes during this process.

In [27]:
def monotonic_increasing_by_one(df, cat):
    nums = df[cat].tolist()
    c = nums[0]
    for i in range(1,len(nums)):
        diff = nums[i] - c
        if diff > 1:
            print("This list is not monotonically increasing by 1 or 0. This occurred at index", i, "where the previous number", c, "is", diff, "less than", nums[i])
            break
        else:
            c = nums[i]
    else:
        print("This list is monotonically increasing by 1 or 0")

In [28]:
print("Checking for Default:")
monotonic_increasing_by_one(observations_default, "Group")
print("Checking for OC-SORT:")
monotonic_increasing_by_one(observations_oc, "Group")

Checking for Default:
This list is monotonically increasing by 1 or 0
Checking for OC-SORT:
This list is monotonically increasing by 1 or 0


Good, now our **Group** attribute has been validated.

### Do passerbys in the same group have the same time?

I've recorded this data where people from the same group would encounter Pepper at the same time. Using this answer in [StackOverflow](https://stackoverflow.com/questions/54518504/check-if-group-contains-same-value-in-pandas), we can do just that:

In [29]:
def same_group_diff_time(df):
    # Returns a list of group numbers that don't have the same time
    a = df.groupby('Group').Time.nunique() > 1
    return a[a].index.tolist()

def fix_group_number_time(df):
    # Given a dataframe, find all instances where group numbers match, but time doesn't, then replace all instances
    # of time mismatch with the first Time of the same group
    for n in same_group_diff_time(df):
        d = df[df.Group==n].iloc[0,0]
        df.loc[df.Group==n,"Time"] = d
    return df

In [30]:
observations_default = fix_group_number_time(observations_default)
observations_oc = fix_group_number_time(observations_oc)

Running these new dataframes through *same_group_diff_time* should yield empty lists:

In [31]:
print("Any more mismatched time for Default?", same_group_diff_time(observations_default))
print("Any more mismatched time for OC-SORT?", same_group_diff_time(observations_oc))

Any more mismatched time for Default? []
Any more mismatched time for OC-SORT? []


And that is our time problem solved and validated.

## Can non-participants be successful, invested, or filled out the survey?

The answer is no. So we will check:

In [32]:
def participant_check(df, name="Default"):
    sis = (df["Success"] + df["Invested"] + df["Survey"]) != 0
    bad = df[(df.Participant==0) & sis]
    if len(bad) > 0: print("You have invalid data for " + name)
    return bad

In [33]:
pcd = participant_check(observations_default)
pco = participant_check(observations_oc, "OC-SORT")

Good, no issues here.

## Is the sum of Survey actually equal to the number of rows from survey dataframes?

In [34]:
print("Survey consistent for Default?", observations_default.Survey.sum() == len(default))
print("Survey consistent for OC-SORT?", observations_oc.Survey.sum() == len(oc))

Survey consistent for Default? True
Survey consistent for OC-SORT? True


## Exporting

In [35]:
os.path.isdir("cleaned")

True

In [36]:
fn = "cleaned"
if not os.path.isdir(fn):
    os.makedirs(fn)
    print("Folder named", fn, "created successfully!")

observations_default.to_csv(os.path.join(fn,"observations_default.csv"), index=False)
observations_oc.to_csv(os.path.join(fn,"observations_oc.csv"), index=False)
default.to_csv(os.path.join(fn,"default.csv"), index=False)
oc.to_csv(os.path.join(fn,"oc.csv"), index=False)